In [57]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [58]:
import pandas as pd
import numpy as np
import env
np.random.seed(123)

In [59]:
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [60]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [61]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
WHERE gender = 'F'
LIMIT 100
'''

employees = pd.read_sql(sql, url)
employees.head()

,emp_no,first_name,last_name
0,10002,Bezalel,Simmel
1,10006,Anneke,Preusig
2,10007,Tzvetan,Zielinski
3,10009,Sumant,Peac
4,10010,Duangkaew,Piveteau


In [62]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
LIMIT 100
'''

title_dept = pd.read_sql(query, url)
title_dept.head()

,title,dept_name
0,Staff,Customer Service
1,Senior Staff,Customer Service
2,Staff,Customer Service
3,Senior Staff,Customer Service
4,Staff,Customer Service


In [63]:
title_dept = pd.read_sql(query, env.get_db_url(user, password, host, 'employees'))
title_dept.head()

,title,dept_name
0,Staff,Customer Service
1,Senior Staff,Customer Service
2,Staff,Customer Service
3,Senior Staff,Customer Service
4,Staff,Customer Service


In [64]:
employees = pd.read_sql('SELECT * FROM employees', url)
employees

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [65]:
titles = pd.read_sql('SELECT * FROM titles', url)
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [66]:
titles.title.value_counts()

Engineer              115003
Staff                 107391
Senior Engineer        97750
Senior Staff           92853
Technique Leader       15159
Assistant Engineer     15128
Manager                   24
Name: title, dtype: int64

In [67]:
titles.title.unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)

In [68]:
len(titles.title.unique())

7

In [69]:
titles.to_date.min()

datetime.date(1985, 3, 1)

In [70]:
titles.to_date.max()

datetime.date(9999, 1, 1)

In [71]:
import datetime

In [72]:
past_employees = titles[titles.to_date != datetime.date(9999, 1, 1)]
past_employees

,emp_no,title,from_date,to_date
3,10004,Engineer,1986-12-01,1995-12-01
6,10005,Staff,1989-09-12,1996-09-12
9,10007,Staff,1989-02-10,1996-02-11
10,10008,Assistant Engineer,1998-03-11,2000-07-31
11,10009,Assistant Engineer,1985-02-18,1990-02-18
...,...,...,...,...
443297,499992,Staff,1987-05-10,1992-05-09
443299,499994,Engineer,1993-02-22,1993-10-27
443301,499996,Engineer,1996-05-13,2002-05-13
443303,499997,Engineer,1987-08-30,1992-08-29


In [73]:
####

In [74]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [75]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [76]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [77]:
users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)
#right join

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [78]:
users.merge(roles, left_on='id', right_on='id', how='outer', indicator=True)
#drop foreign keys

,id,name_x,role_id,name_y,_merge
0,1,bob,1.0,admin,both
1,2,joe,2.0,author,both
2,3,sally,3.0,reviewer,both
3,4,adam,3.0,commenter,both
4,5,jane,NaN,NaN,left_only
5,6,mike,NaN,NaN,left_only


In [79]:
from pydataset import data


In [80]:
mpg=data('mpg')
data('mpg', show_doc=True)

#234 rows, 11 columns
#38 models of cars

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [81]:
mpg.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [82]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [83]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [84]:
mpg.rename(columns={'cty':'city', 'class':'clss'},inplace = True)
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,clss
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [85]:
mpg.model.nunique()
#alternative way to count unique values

38

In [86]:
len(mpg.manufacturer.unique())
#shows how many manufacturers there are

15

In [87]:
len(mpg.value_counts('model'))
#shows how many models there are

38

In [88]:
mpg['mileage_difference']=(mpg.hwy-mpg.city)
#mileage difference column

In [89]:
mpg['avg_mileage']=(mpg.city+mpg.hwy)/2
#avg mileage column

In [90]:
#modify elements of trans col so i can find only the autos in a boolean
mpg['trans'] = mpg.trans.str.replace(r"\(.*\)","")
mpg

C:\Users\Timmy\AppData\Local\Temp\ipykernel_16236\211151484.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  mpg['trans'] = mpg.trans.str.replace(r"\(.*\)","")


,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,clss,mileage_difference,avg_mileage
1,audi,a4,1.8,1999,4,auto,f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual,f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual,f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto,f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto,f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto,f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual,f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto,f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual,f,18,26,p,midsize,8,22.0


In [91]:
mpg['is_automatic'] = mpg.trans == 'auto'
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,clss,mileage_difference,avg_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto,f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual,f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual,f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto,f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto,f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto,f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual,f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto,f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual,f,18,26,p,midsize,8,22.0,False


In [92]:
mpg.sort_values(by='avg_mileage').nlargest(1, 'avg_mileage')
#best avg mileage

,manufacturer,model,displ,year,cyl,trans,drv,city,hwy,fl,clss,mileage_difference,avg_mileage,is_automatic
222,volkswagen,new beetle,1.9,1999,4,manual,f,35,44,d,subcompact,9,39.5,False


In [93]:
mpg.groupby('manufacturer').avg_mileage.mean().nlargest(n=1)
#alt method for manufacturer with best overall avg

manufacturer
honda    28.5
Name: avg_mileage, dtype: float64

In [94]:
mpg.groupby('is_automatic').avg_mileage.mean().nlargest(n=2)
#shows avg mpgs across the automatic and manual trans

is_automatic
False    22.227273
True     19.130573
Name: avg_mileage, dtype: float64

In [95]:
mpg[mpg.trans == 'auto'].count()
#157 autos with automatic trans

manufacturer          157
model                 157
displ                 157
year                  157
cyl                   157
trans                 157
drv                   157
city                  157
hwy                   157
fl                    157
clss                  157
mileage_difference    157
avg_mileage           157
is_automatic          157
dtype: int64

In [96]:
mpg[mpg.trans == 'manual'].count()
#77 autos with manual trans

manufacturer          77
model                 77
displ                 77
year                  77
cyl                   77
trans                 77
drv                   77
city                  77
hwy                   77
fl                    77
clss                  77
mileage_difference    77
avg_mileage           77
is_automatic          77
dtype: int64

In [97]:
autotrans_avgmpg = mpg[mpg.trans == 'auto'].mean()
autotrans_avgmpg
#on average the automatic autos have an avg mpg of 19.13

C:\Users\Timmy\AppData\Local\Temp\ipykernel_16236\4266081611.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  autotrans_avgmpg = mpg[mpg.trans == 'auto'].mean()


displ                    3.719745
year                  2003.757962
cyl                      6.203822
city                    15.968153
hwy                     22.292994
mileage_difference       6.324841
avg_mileage             19.130573
is_automatic             1.000000
dtype: float64

In [98]:
mantrans_avgmpg = mpg[mpg.trans == 'manual'].mean()
mantrans_avgmpg
#on average the automatic autos have an avg mpg of 22.22
#manuals have better avg mpgs

C:\Users\Timmy\AppData\Local\Temp\ipykernel_16236\2703569575.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mantrans_avgmpg = mpg[mpg.trans == 'manual'].mean()


displ                    2.966234
year                  2002.974026
cyl                      5.246753
city                    18.675325
hwy                     25.779221
mileage_difference       7.103896
avg_mileage             22.227273
is_automatic             0.000000
dtype: float64

In [99]:
### exercise set 3

In [103]:
url1 = f'mysql+pymysql://{user}:{password}@{host}/chipotle'


In [105]:
chipotle = pd.read_sql('SELECT * FROM orders', url1)

In [107]:
chipotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


In [ ]:
#item price is object

In [128]:
chipotle.item_price=chipotle.item_price.str.replace('$','')
#remove special char from item price, change type to float



AttributeError: Can only use .str accessor with string values!

In [125]:
chipotle

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75


In [127]:
chipotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  4622 non-null   int64  
 1   order_id            4622 non-null   int64  
 2   quantity            4622 non-null   int64  
 3   item_name           4622 non-null   object 
 4   choice_description  4622 non-null   object 
 5   item_price          4622 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 216.8+ KB


In [130]:
chipotle['order_total'] = chipotle.quantity * chipotle.item_price
chipotle.head()
#new column with order totals

,id,order_id,quantity,item_name,choice_description,item_price,order_total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,33.96


In [135]:
chipotle.item_name.value_counts().nlargest(n=3)
#grabbed the unique item names chained with value counts and nlargest to grab top three

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

In [139]:
chipotle.item_name.unique()

array(['Chips and Fresh Tomato Salsa', 'Izze', 'Nantucket Nectar',
       'Chips and Tomatillo-Green Chili Salsa', 'Chicken Bowl',
       'Side of Chips', 'Steak Burrito', 'Steak Soft Tacos',
       'Chips and Guacamole', 'Chicken Crispy Tacos',
       'Chicken Soft Tacos', 'Chicken Burrito', 'Canned Soda',
       'Barbacoa Burrito', 'Carnitas Burrito', 'Carnitas Bowl',
       'Bottled Water', 'Chips and Tomatillo Green Chili Salsa',
       'Barbacoa Bowl', 'Chips', 'Chicken Salad Bowl', 'Steak Bowl',
       'Barbacoa Soft Tacos', 'Veggie Burrito', 'Veggie Bowl',
       'Steak Crispy Tacos', 'Chips and Tomatillo Red Chili Salsa',
       'Barbacoa Crispy Tacos', 'Veggie Salad Bowl',
       'Chips and Roasted Chili-Corn Salsa',
       'Chips and Roasted Chili Corn Salsa', 'Carnitas Soft Tacos',
       'Chicken Salad', 'Canned Soft Drink', 'Steak Salad Bowl',
       '6 Pack Soft Drink', 'Chips and Tomatillo-Red Chili Salsa', 'Bowl',
       'Burrito', 'Crispy Tacos', 'Carnitas Crispy Tacos